In [27]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define a ticker and a date range for your data
ticker = 'SPY'
start_date = '2000-01-01'
end_date = '2025-07-31'
window = 5
lookback_range = [3,21]

# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
data = data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
data = data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
data.insert(1, 'Ticker', ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2000-01-03,SPY,92.142509,93.924380,91.152581,93.924380,8164300
1,2000-01-04,SPY,88.539185,91.271387,88.469890,90.934811,8089800
2,2000-01-05,SPY,88.697571,89.667700,86.955297,88.657974,12177900
3,2000-01-06,SPY,87.272102,89.647931,87.272102,88.460017,6227200
4,2000-01-07,SPY,92.340515,92.340515,88.737176,88.895565,8066500
...,...,...,...,...,...,...,...
6427,2025-07-24,SPY,634.419983,636.150024,633.989990,634.599976,71307100
6428,2025-07-25,SPY,637.099976,637.580017,634.840027,635.090027,56865400
6429,2025-07-28,SPY,636.940002,638.039978,635.539978,637.479980,54917100
6430,2025-07-29,SPY,635.260010,638.669983,634.340027,638.349976,60556300


In [28]:
def calculate_sma(df, lookback):
    """
    Calculates a simple moving average for a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'close' column.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the moving average.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_sma = df.copy()

    # Calculate the simple moving average
    df_sma['SMA'] = df_sma['Close'].rolling(window=lookback).mean()

    # Calculate the difference between the SMA and the Close price
    df_sma['SMA_sign'] = np.sign(df_sma['Close'] - df_sma['SMA'])

    return df_sma



In [29]:
def analyze_sma_changes(df, lookback):
    """
    Analyzes the impact of Simple Moving Average sign changes on price differences.

    Args:
        df (pd.DataFrame): The input DataFrame with historical price data.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: A DataFrame containing the sum of Next_Close_Diff at SMA sign changes by year,
                      including the lookback value.
    """
    # Use the existing calculate_sma function
    df_with_sma = calculate_sma(df=df.copy(), lookback=lookback)

    # 1. Create a Series of the previous row's sign value.
    previous_sign = df_with_sma['SMA_sign'].shift(1)

    # 2. Create a boolean mask that is True when the sign changes AND the previous sign is not NaN.
    sign_changed_mask = (df_with_sma['SMA_sign'] != previous_sign) & (~previous_sign.isna())

    # 3. Filter the DataFrame using the boolean mask and explicitly create a copy.
    sign_changes_only = df_with_sma[sign_changed_mask].copy()

    # Now, add the 'Next_Close_Diff' column to the copy.
    sign_changes_only.loc[:, 'Next_Close_Diff'] = sign_changes_only['Close'].shift(-1) - sign_changes_only['Close']

    # Ensure 'Date' column is in datetime format
    sign_changes_only['Date'] = pd.to_datetime(sign_changes_only['Date'])

    # Extract the year from the 'Date' column
    sign_changes_only['Year'] = sign_changes_only['Date'].dt.year

    # Filter by SMA_sign = 1
    df_filtered_sign_changes = sign_changes_only[sign_changes_only['SMA_sign'] == 1].copy()

    # Group by year and sum the 'Next_Close_Diff'
    yearly_next_close_diff_sum = df_filtered_sign_changes.groupby('Year')['Next_Close_Diff'].sum().reset_index()

    # Add the 'Lookback' column with the specified lookback value
    yearly_next_close_diff_sum['Lookback'] = lookback

    # Return the DataFrame with columns in the desired order: Year, Lookback, Next_Close_Diff
    return yearly_next_close_diff_sum[['Year', 'Lookback', 'Next_Close_Diff']]

In [30]:
# Initialize an empty list to store the results from each lookback value
results_list = []

# Iterate through each value in the specified range
for lookback_value in range(lookback_range[0], lookback_range[1] + 1):
    # Calculate SMA changes for the current lookback value
    df_sma_result = analyze_sma_changes(df=data.copy(), lookback=lookback_value)
    # Append the result to the list
    results_list.append(df_sma_result)

# Concatenate all the DataFrames in the list into a single DataFrame
all_sma_results = pd.concat(results_list, ignore_index=True)

# Print the resulting DataFrame
print("DataFrame with Simple Moving Average analysis for different lookback values:")
display(all_sma_results)

DataFrame with Simple Moving Average analysis for different lookback values:


,Year,Lookback,Next_Close_Diff
0,2000,3,-11.280655
1,2001,3,-6.904366
2,2002,3,-17.611702
3,2003,3,1.009567
4,2004,3,-1.783371
...,...,...,...
489,2021,21,38.774384
490,2022,21,-66.087708
491,2023,21,71.170685
492,2024,21,53.843353


In [31]:
# Pivot the DataFrame
pivoted_sma_results = all_sma_results.pivot(index='Year', columns='Lookback', values='Next_Close_Diff')

# Display the pivoted DataFrame
print("Pivoted DataFrame with Lookback as columns, Year as rows, and Next_Close_Diff as values:")
display(pivoted_sma_results)

Pivoted DataFrame with Lookback as columns, Year as rows, and Next_Close_Diff as values:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2000,-11.280655,-12.530174,-14.567146,-11.931602,-8.747513,-13.146423,-14.186691,-16.294342,-14.697350,-16.646248,-14.356850,-14.166985,-17.188461,-12.938904,-12.767105,-16.256584,-15.593727,-17.075867,-17.075867
2001,-6.904366,-6.129425,-2.343147,-2.036804,-1.623184,-5.621758,-1.462593,-2.591843,-5.556313,-7.640541,-8.758011,-7.506203,-8.682602,-5.069702,-0.438332,0.366798,-1.274147,1.253090,0.080559
2002,-17.611702,-15.134869,-10.063030,-10.402420,-14.196156,-14.619049,-13.253349,-13.774624,-13.843468,-8.939213,-5.183769,-0.575813,2.811592,1.766693,1.799202,3.871002,3.276348,1.435368,1.435368
2003,1.009567,3.206463,6.269905,10.897259,10.782871,9.088535,9.300648,9.759228,9.171230,7.166836,7.206009,5.451885,6.848354,7.996532,6.816090,7.698723,8.140076,8.158085,9.586334
2004,-1.783371,0.019630,-1.380180,2.532776,3.049850,2.856422,4.347893,4.739876,2.665878,3.716232,5.112480,5.119232,4.634293,3.079239,1.942520,3.197227,3.197227,3.492889,3.486221
2005,-6.167824,-6.754829,-4.597160,-1.626373,-1.449677,-4.047417,-3.603035,-3.413406,-1.793533,-1.358231,-0.625847,-0.051315,-1.335312,0.580124,1.844963,0.580711,0.519066,0.937065,0.827316
2006,2.002701,2.340485,2.788216,3.605835,4.846596,4.747421,4.327698,3.302879,2.253555,0.601562,1.491013,3.482407,4.126511,1.842659,2.738800,5.261200,4.488083,5.965073,4.007271
2007,-1.498596,3.736649,-5.033951,-3.313942,-0.228325,2.017014,3.568756,2.195984,1.225502,-0.713600,-0.194695,-2.213432,-2.432961,-2.524857,-3.523491,-2.082550,-1.417191,-1.417191,-1.080505
2008,-41.780399,-51.492619,-51.439255,-57.213989,-51.504246,-55.909321,-42.990269,-40.593185,-30.644161,-22.620571,-29.711884,-24.072525,-24.437561,-26.848801,-28.485104,-25.135433,-25.618824,-24.525661,-26.297108


In [32]:
# Ensure 'Date' column in the initial 'data' DataFrame is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract the year from the 'Date' column
data['Year'] = data['Date'].dt.year

# Group by year and get the first and last close prices
yearly_price_change = data.groupby('Year')['Close'].agg(['first', 'last'])

# Calculate the difference between the last and first close price for each year
yearly_price_change['Yearly_Change'] = yearly_price_change['last'] - yearly_price_change['first']

# Drop the 'first' and 'last' columns
yearly_price_change = yearly_price_change.drop(columns=['first', 'last'])

# Display the result
print("Difference between the last and first close price of each year:")
display(yearly_price_change)

Difference between the last and first close price of each year:


,Yearly_Change
Year,
2000,-8.154060
2001,-8.355286
2002,-16.794479
2003,14.507263
2004,8.002777
2005,4.370407
2006,12.176880
2007,5.330307
2008,-37.821625


In [33]:
# Join the yearly_price_change DataFrame with the yearly_next_close_diff_sum Series on the 'Year' index
comparison_df = yearly_price_change.join(pivoted_sma_results)

# Display the combined DataFrame, excluding the 'first' and 'last' columns
display(comparison_df)

,Yearly_Change,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,,
2000,-8.154060,-11.280655,-12.530174,-14.567146,-11.931602,-8.747513,-13.146423,-14.186691,-16.294342,-14.697350,-16.646248,-14.356850,-14.166985,-17.188461,-12.938904,-12.767105,-16.256584,-15.593727,-17.075867,-17.075867
2001,-8.355286,-6.904366,-6.129425,-2.343147,-2.036804,-1.623184,-5.621758,-1.462593,-2.591843,-5.556313,-7.640541,-8.758011,-7.506203,-8.682602,-5.069702,-0.438332,0.366798,-1.274147,1.253090,0.080559
2002,-16.794479,-17.611702,-15.134869,-10.063030,-10.402420,-14.196156,-14.619049,-13.253349,-13.774624,-13.843468,-8.939213,-5.183769,-0.575813,2.811592,1.766693,1.799202,3.871002,3.276348,1.435368,1.435368
2003,14.507263,1.009567,3.206463,6.269905,10.897259,10.782871,9.088535,9.300648,9.759228,9.171230,7.166836,7.206009,5.451885,6.848354,7.996532,6.816090,7.698723,8.140076,8.158085,9.586334
2004,8.002777,-1.783371,0.019630,-1.380180,2.532776,3.049850,2.856422,4.347893,4.739876,2.665878,3.716232,5.112480,5.119232,4.634293,3.079239,1.942520,3.197227,3.197227,3.492889,3.486221
2005,4.370407,-6.167824,-6.754829,-4.597160,-1.626373,-1.449677,-4.047417,-3.603035,-3.413406,-1.793533,-1.358231,-0.625847,-0.051315,-1.335312,0.580124,1.844963,0.580711,0.519066,0.937065,0.827316
2006,12.176880,2.002701,2.340485,2.788216,3.605835,4.846596,4.747421,4.327698,3.302879,2.253555,0.601562,1.491013,3.482407,4.126511,1.842659,2.738800,5.261200,4.488083,5.965073,4.007271
2007,5.330307,-1.498596,3.736649,-5.033951,-3.313942,-0.228325,2.017014,3.568756,2.195984,1.225502,-0.713600,-0.194695,-2.213432,-2.432961,-2.524857,-3.523491,-2.082550,-1.417191,-1.417191,-1.080505
2008,-37.821625,-41.780399,-51.492619,-51.439255,-57.213989,-51.504246,-55.909321,-42.990269,-40.593185,-30.644161,-22.620571,-29.711884,-24.072525,-24.437561,-26.848801,-28.485104,-25.135433,-25.618824,-24.525661,-26.297108


In [34]:
# Calculate the sum and standard deviation of each column in the comparison_df
grand_totals = comparison_df.sum()
standard_deviations = comparison_df.std()

# Calculate the Mean Absolute Deviation for each column
mean_absolute_deviations = comparison_df.apply(lambda x: (x - x.mean()).abs().mean())

# Combine the metrics into a single DataFrame for display
summary_df = pd.DataFrame({
    'Grand Total': grand_totals,
    'Standard Deviation': standard_deviations,
    'Mean Absolute Deviation': mean_absolute_deviations
})

# Display the summary DataFrame
print("Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:")
display(summary_df)

Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:


,Grand Total,Standard Deviation,Mean Absolute Deviation
Yearly_Change,537.775261,43.960125,31.610253
3,108.436298,27.723224,17.256445
4,136.468914,33.943672,20.653725
5,95.655270,33.242604,20.610997
6,131.161781,33.083361,20.751296
7,205.538074,28.933924,19.759530
8,215.569435,28.681041,20.146037
9,210.136417,27.534647,19.075547
10,264.726990,30.720848,20.577500
11,228.066383,31.711192,20.833765
